In [185]:
import os
import plaid
import requests
import datetime
import json
import pandas as pd
%matplotlib inline

In [186]:
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [187]:
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_SECRET = os.getenv('PLAID_SECRET')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_ENV = os.getenv('PLAID_ENV', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')

# Plaid Access Token

In this section, you will use the plaid-python api to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:
1. Create a client to connect to paid
2. Use the client to generate a public token and request the following items: 
['transactions', 'income', 'assets']
3. Exchange the public token for an access token
4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [188]:
INSITUTION_ID = "ins_109508"

In [189]:
# YOUR CODE HERE
client = plaid.Client(client_id=PLAID_CLIENT_ID, secret=PLAID_SECRET, public_key=PLAID_PUBLIC_KEY, environment="sandbox")
print(client)

### 2. Generate a public token

In [190]:
# YOUR CODE HERE
generate_tkn_response = client.Sandbox.public_token.create(INSITUTION_ID, ["transactions", "income", "assets"])


### 3. Exchange the public token for an access token

In [191]:
# YOUR CODexchangeE HERE
exchange_token = client.Item.public_token.exchange(generate_tkn_response["public_token"])
#store token as varible 
access_token = exchange_token ["access_token"]
print(access_token)

access-sandbox-d1445515-451f-4472-8be4-32831b8742b0


### 4. Fetch Accounts

In [192]:
# YOUR CODE HERE
client.Accounts.get(access_token)


{'accounts': [{'account_id': 'PDZzlelX4KH6RAaZGjvXiNZZv9oGWWi7PBkZy',
   'balances': {'available': 100,
    'current': 110,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '0000',
   'name': 'Plaid Checking',
   'official_name': 'Plaid Gold Standard 0% Interest Checking',
   'subtype': 'checking',
   'type': 'depository'},
  {'account_id': 'jnmEWZWXdGhQW6ek8ZBNHw99GVxqzzI1VzMqK',
   'balances': {'available': 200,
    'current': 210,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '1111',
   'name': 'Plaid Saving',
   'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
   'subtype': 'savings',
   'type': 'depository'},
  {'account_id': '7D5qKQK3j4HKJegbw1mRU7WWaJP5eecgoAlLy',
   'balances': {'available': None,
    'current': 1000,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '2222',
   'name': 'Plaid CD',
   'officia

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [193]:
# YOUR CODE HEREat
start_date = "{:%Y-%m-%d}".format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = "{:%Y-%m-%d}". format(datetime.datetime.now())
#transactions for date range
transaction_response = client.Transactions.get(access_token,start_date,end_date)
#print JSON output
transactions_lists = json.dumps(transaction_response["transactions"], indent=4, sort_keys=True)
print(transactions_lists)

ItemError: the requested product is not yet ready. please provide a webhook or try the request again later

### 2. Print the categories for each transaction

In [ ]:
# YOUR CODE HERE
transactions = transaction_response["transactions"]
all_categories = []
for tran in transactions:
    category = tran["category"][0]
    all_categories.append(category)
    print(category)

### 3. Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

(For categories with more than one label, just use the first category label in the list)

In [ ]:
# YOUR CODE HERE
transactions = transactions_response["transactions"]

all_dates = []
all_seller_name = []
all_amount = []
all_categories = []

for tran in transactions:
    category = tran["category"][0]
    all_categories.append(category)
    all_dates.append(tran["date"])
    all_seller_name.append(tran["name"])
    all_amount.append(tran["amount"])
    
for i in range(len(all_dates)):
    print(f"{all_dates[i]} {all_seller_name[i]} \t ${all_amount[i]} Cat: {all_categories[i]}")

df= pd.DataFrame (list(zip(all_dates,all_seller_name,all_amount,all_categories)),
                  columns =["Date", "Seller", "Amount", "Category"])
df.head()

### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [ ]:
# YOUR CODE HERE
df["Seller"].astype("str")
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date",inplace=True
#print(df.dtypes)
#df.head()

---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [ ]:
# YOUR CODE HERE

---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calulate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
# YOUR CODE HERE

### Calculate the expenses per month

In [ ]:
# YOUR CODE HERE